
## Import 

In [ ]:
import os, sys, pickle

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
# from xgboost import XGBClassifier



# import shap

## Load data

In [ ]:
train = "./data/cleaned/df_train.csv"
df_train = pd.read_csv(train)
df_train.head()

In [ ]:
test = "./data/cleaned/df_test.csv"
df_test = pd.read_csv (test)

In [ ]:
## Model Load

In [ ]:
with open("./models/model.pk", "rb") as f : 

    model = pickle.load(f)

In [ ]:
model

# Test model

In [ ]:
df_train

In [ ]:
X_train_PassengerId = df_train.PassengerId.values
y_train = df_train.Survived.values
X_train = df_train.drop(columns=["Survived","PassengerId" ], inplace=False)
X_train.head()

In [ ]:
model.predict(X_train)

In [ ]:
model.score(X_train, y_train)

# EXPLAIN

In [ ]:
feat = pd.Series(model.feature_importances_, index=X_train.columns)# .sort_values(ascending=False)
px.bar(feat)

In [ ]:

feat = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
px.bar(feat)

In [ ]:
explain = pd.Series(model.feature_importances_, index= model.feature_names_in_)
explain

In [ ]:
explain = {k: v.round(2) for v, k in zip(model.feature_importances_, model.feature_names_in_)}
explain


In [ ]:
explainer = shap.TreeExplainer(model) # Explainer
explainer

In [ ]:
with open("./explainers/shap.pk", "wb") as f : 
    pk = pickle.dumps(explainer)
    f.write(pk)

In [ ]:
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar",show=True)

In [ ]:
shap.summary_plot(shap_values, X_train)


In [ ]:
shap.initjs()


In [ ]:
X_train.iloc[0]

In [ ]:
df_train.iloc[0]

In [ ]:
def extract_vect(PassengerId, _df) : 

    X = _df.loc[_df.PassengerId == PassengerId]
    assert len(X) == 1

    Passenger_id =  X.PassengerId[0]
    Survived = X.Survived[0]
    X = X.drop(columns = ["PassengerId", "Survived"])
    ser = X.iloc[0].to_dict()

    return ser, Passenger_id, Survived


In [ ]:
__X, __Passenger_id, __Survived = extract_vect(848, df_train)

In [ ]:
__X

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], pd.Series(__X))

In [ ]:
pd.DataFrame(shap_values[0])

In [ ]:
pd.DataFrame(shap_values[1])

In [ ]:
pd.DataFrame(shap_values[2])

In [ ]:
"""
shap.summary_plot(shap_values, X_train, plot_type="bar",show=False)
plt.savefig('grafic.png')
"""

In [ ]:
"""
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
"""

explainer = shap.TreeExplainer(model, X_train)
shap_values = explainer(X_train)
shap.plots.waterfall(shap_values[0])
"""

In [ ]:
shap.plots.waterfall(shap_values[0])

## Save